<img src="img/header1.png"></img>

<h1 style="text-align: left;">ToT: PySpark Basics to Inter./Adv. in ± 15 Minutes</h1>
By: Imam Cholissodin | Dosen Fakultas Ilmu Komputer (FILKOM), Universitas Brawijaya (UB)

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-koding-dasar").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

sc = spark.sparkContext

In [3]:
spark

In [4]:
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-koding-dasar>

## Contoh Koding Spark dengan Resilient Distributed Datasets (RDD)

![title](ilustrasi-rdd.png)

Semuanya berjalan di Memory, sehingga bisa lebih cepat dari pada Map Reduce dari Hadoop

In [7]:
# Cek banyak node
sc._jsc.sc().getExecutorMemoryStatus().size()

3

In [37]:
# Koding Spark RDD | Map
def add1(x): 
    return x+5

raw_data = sc.parallelize([1,2,3])
rdd = raw_data.map(lambda x: add1(x))
#print(rdd.take(3))
print(rdd.collect())

[6, 7, 8]


In [42]:
# Koding Spark RDD | Filter
def isGanjil(x): 
    return x%2==1

raw_data = sc.parallelize(range(1,6))
rdd = raw_data.filter(lambda x: isGanjil(x))
#print(rdd.take(3))
print(rdd.collect())

[1, 3, 5]


In [44]:
# Koding Spark RDD | Reduce
def jumlahkan(x,y): 
    return x+y

raw_data = sc.parallelize(range(1,6))
#rdd = raw_data.reduce(lambda x,y:x+y)

# 1 + 2 + 3 + 4 + 5 = 15
rdd = raw_data.reduce(lambda x,y:jumlahkan(x,y))
print(rdd)

15


In [45]:
# Koding Spark RDD | Lambda Spark (tanpa memberikan nama fungsi)
raw_data = sc.parallelize(range(1,6))
rdd = raw_data.reduce(lambda x,y:x+y)
print(rdd)

15


In [49]:
# python standar
(lambda x:2*x)(3)

6

In [52]:
# set dalam dalam bentuk user define function (UDF), misal dalam variabel f
f = lambda x:2*x

In [53]:
# Koding Spark RDD | Mengalikan setiap elemen raw_data dengan 2
raw_data = sc.parallelize([1,2,3])
rdd=raw_data.map(lambda x:2*x)
print(rdd.collect())

[2, 4, 6]


In [54]:
# Koding Spark RDD | Mengalikan setiap elemen raw_data dengan 2
raw_data = sc.parallelize([1,2,3])
rdd=raw_data.map(f)
print(rdd.collect())

[2, 4, 6]


In [56]:
# Koding Spark RDD | Mengambil elemen awal dari tiap pasang raw_data
raw_data = sc.parallelize([(1,2),(3,4),(5,6)])
rdd=raw_data.map(lambda x:x[0])
print(rdd.collect())

[1, 3, 5]


In [59]:
# Koding Spark RDD | Menjumlahkan semua elemen pada raw_data
raw_data = sc.parallelize([1,2,3])
rdd = raw_data.reduce(lambda x,y:x+y)
print(rdd)

6


In [60]:
# Koding Spark RDD | Menjumlahkan hanya elemen pertama tiap pasangan raw_data
raw_data = sc.parallelize([(1,2),(3,4),(5,6)])
rdd=raw_data.map(lambda x:x[0]).reduce(lambda x,y:x+y)
print(rdd)

9


## Tambahan 1 : Koding dengan Spark RDD

In [32]:
!curl https://raw.githubusercontent.com/imamcs19/ScrapingNothers-toOneDataNothers/master/nilai-uas.csv > nilai-uas-mhs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    57  100    57    0     0    553      0 --:--:-- --:--:-- --:--:--   553


In [79]:
# Load Data
# data = sc.parallelize([("Mhs 1", 95), ("Mhs 2", 70), ("Mhs 3", 85), ("Mhs 4", 100)])
data = sc.textFile("nilai-uas-mhs.csv") \
    .map(lambda line: line.split(",")) \
    .zipWithIndex().filter(lambda baris: baris[1] > 0)\
    .map(lambda x:x[0])\
    .map(lambda x:(x[0],int(x[1])))

# menampilkan data awal, yaitu mahasiswa dan nilai uas-nya
print("Menampilkan data awal, yaitu mahasiswa dan nilai uas-nya:")
print(data.collect())


Menampilkan data awal, yaitu mahasiswa dan nilai uas-nya:
[('Mhs 1', 95), ('Mhs 2', 70), ('Mhs 3', 85), ('Mhs 4', 100)]


In [81]:
print("Menampilkan hanya nama mahasiswa:")
hasil1=data.map(lambda x: x[0]).collect()
print(hasil1)

Menampilkan hanya nama mahasiswa:
['Mhs 1', 'Mhs 2', 'Mhs 3', 'Mhs 4']


In [82]:
print("Menampilkan hanya nilai uas semua mahasiswa:")
hasil2=data.map(lambda t: t[1]).collect()
print(hasil2)

Menampilkan hanya nilai uas semua mahasiswa:
[95, 70, 85, 100]


In [16]:
print("Menampilkan total nilai uas semua mahasiswa-nya:")
raw_data = sc.parallelize(hasil2)
hasil3 = raw_data.reduce(lambda x,y:x+y)
print(hasil3)

Menampilkan total nilai uas semua mahasiswa-nya:
350


In [17]:
print("Menampilkan total nilai uas semua mahasiswa-nya:")
hasil31=data.map(lambda t: (1,int(t[1]))).reduceByKey(lambda x,y: x+y).collect()
print(hasil31)

Menampilkan total nilai uas semua mahasiswa-nya:
[(1, 350)]


In [19]:
print("Menampilkan nilai minimum dari semua mahasiswa-nya:")
raw_data = sc.parallelize(hasil2)
hasil4 = raw_data.reduce(lambda x,y:min(x,y))
print(hasil4)

Menampilkan nilai minimum dari semua mahasiswa-nya:
70


In [21]:
print("Menampilkan nilai maksimum dari semua mahasiswa-nya:")
raw_data = sc.parallelize(hasil2)
hasil5 = raw_data.reduce(lambda x,y:max(x,y))
print(hasil5)

Menampilkan nilai minimum dari semua mahasiswa-nya:
100


In [26]:
print("Menampilkan banyak mahasiswa:")
hasil6 = data.map(lambda t: 1).reduce(lambda x,y:x+y)
print(hasil6)

Menampilkan banyak mahasiswa:
4


In [28]:
print("Menampilkan nilai rata-rata dari semua nilai uas mahasiswa:")
hasil7=data.map(lambda x:x[1]).reduce(lambda x,y:x+y) / data.map(lambda x:1).reduce(lambda x,y:x+y)
print(hasil7)

Menampilkan nilai rata-rata dari semua nilai uas mahasiswa:
87.5


## Tambahan 2 : Koding dengan Spark SQL

In [100]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
#from pyspark.sql import functions as F    dimana min => F.min, max => F.max
from pyspark.sql.functions import min
from pyspark.sql.functions import max

# cara convert rdd ke df
# rdd2df=data.toDF(["Nama","Nilai UAS"])

# atau
rdd2df = spark.read.csv(path="nilai-uas-mhs.csv", sep=",", header=True)

# convert kolom "Nilai UAS" menjadi integer
rdd2df = rdd2df.withColumn("Nilai UAS", col("Nilai UAS").cast('int'))

# df = spark.createDataFrame([("Mhs 1", 95), ("Mhs 2", 70), ("Mhs 3", 85), ("Mhs 4", 100)], ["Nama", "Nilai UAS"])

# menampilkan data awal, yaitu mahasiswa dan nilai uas-nya
print("Menampilkan data awal, yaitu mahasiswa dan nilai uas-nya:")
rdd2df.show()

print("\n")

print("Menampilkan nilai uas yang paling minimal cara ke-1:")
minNilaiUAS = rdd2df \
    .groupBy() \
    .min('Nilai UAS') \
    .select(col("min(Nilai UAS)").alias("min Nilai UAS"))
    
minNilaiUAS.show()

print("\n")

print("Menampilkan nilai uas yang paling minimal cara ke-2:")
hasil=df.groupBy().agg(min(col('Nilai UAS')))
hasil.show()

Menampilkan data awal, yaitu mahasiswa dan nilai uas-nya:
+-----+---------+
| Nama|Nilai UAS|
+-----+---------+
|Mhs 1|       95|
|Mhs 2|       70|
|Mhs 3|       85|
|Mhs 4|      100|
+-----+---------+



Menampilkan nilai uas yang paling minimal cara ke-1:
+-------------+
|min Nilai UAS|
+-------------+
|           70|
+-------------+



Menampilkan nilai uas yang paling minimal cara ke-2:
+--------------+
|min(Nilai UAS)|
+--------------+
|            70|
+--------------+



In [103]:
from pyspark.sql.functions import col
from pyspark.sql.functions import mean
from pyspark.sql.functions import sum

# data nilai uas mahasiswa
df = spark.createDataFrame(
[("Mhs 1", "P", 95), ("Mhs 2", "L", 70), 
("Mhs 3", "P", 85), ("Mhs 4", "L", 100)], 
["Nama", "Jenis Kelamin", "Nilai UAS"])

print("Data nilai uas mahasiswa:")
df.show()

print("\n")

print("Menampilkan rata-rata nilai uas berdasarkan JK:")
df.groupBy('Jenis Kelamin').agg(mean(col('Nilai UAS'))).show()

print()

Data nilai uas mahasiswa:
+-----+-------------+---------+
| Nama|Jenis Kelamin|Nilai UAS|
+-----+-------------+---------+
|Mhs 1|            P|       95|
|Mhs 2|            L|       70|
|Mhs 3|            P|       85|
|Mhs 4|            L|      100|
+-----+-------------+---------+



Menampilkan rata-rata nilai uas berdasarkan JK:
+-------------+--------------+
|Jenis Kelamin|avg(Nilai UAS)|
+-------------+--------------+
|            L|          85.0|
|            P|          90.0|
+-------------+--------------+


Contoh pada Data Pajak Perusahaan:
Data Pajak Perusahaan:
+---------------+----------------+------------------+
|Nama Perusahaan|Jenis Perusahaan|Pajak (Rp x 1000K)|
+---------------+----------------+------------------+
|   Perusahaan 1|          Tipe B|              1000|
|   Perusahaan 2|          Tipe C|               500|
|   Perusahaan 3|          Tipe A|             10000|
|   Perusahaan 4|          Tipe B|              1000|
+---------------+----------------+---------

In [10]:
# Ref:
# [1] Cholissodin, I., Riyandani, E., 2016, Analisis Big Data, Fakultas Ilmu Komputer, Universitas Brawijaya, Malang.
# [2] Cholissodin, I. and Supianto, A. A. 2019. Enhancement Full Open Source Hadoop Distribution Universal Big Data Up Projects (UBig) From Education To Enterprise. 2019 International Conference on Sustainable Information Engineering and Technology (SIET), Lombok, Indonesia, 2019, pp. 90-93, doi: 10.1109/SIET48054.2019.8986040.
# [3] Miao, K., Li, J., Hong, W. & Chen, M. 2020. A Microservice-Based Big Data Analysis Platform for Online Educational Applications. Annual Review of Anthropology, 2020, [ "6929750" ]. Available from: https://doi.org/10.1146/annurev.anthro.33.070203.144008
# [4] Roy, S., et al. 2017. IoT, big data science & analytics, cloud computing and mobile app based hybrid system for smart agriculture. 2017 8th Annual Industrial Automation and Electromechanical Engineering Conference (IEMECON), Bangkok, 2017, pp. 303-304, doi: 10.1109/IEMECON.2017.8079610.
# [5] Dabek, F. 2016. Leveraging Big Data to Provide a Web Service That Provides the Likelihood of Developing Psychological Conditions after a Concussion. 2016 IEEE International Conference on Mobile Services (MS), San Francisco, CA, 2016, pp. 160-165, doi: 10.1109/MobServ.2016.32.
# [6] Andy A. 2016. Cloud Computing Part 5: SaaS (Software as a Service). Available at: https://andypi.co.uk/2016/05/23/cloud-computing-part-5-saas-software-as-a-service/ (Accessed on July 30, 2020).
# [7] Putra, N. A., Putri, A. T., Prabowo, D. A., Surtiningsih, L., Arniantya, R., Cholissodin, I. 2017. Klasifikasi Sepeda Motor Berdasarkan Karakteristik Konsumen Dengan Metode K-Nearest Neighbour Pada Big Data Menggunakan Hadoop Single Node Cluster. Jurnal Teknologi Informasi dan Ilmu Komputer (JTIIK) FILKOM UB Vol. 4 No. 2, 81-86.
# [8] Naveen, N. 2020. Hadoop MapReduce – Key Features & Highlights. Available at: https://intellipaat.com/blog/tutorial/big-data-and-hadoop-tutorial/hadoop-mapreduce-key-features-highlights/ (Accessed on March 8, 2020).
# [9] stackchief. 2017. MapReduce Quick Explanation. Available at: https://www.stackchief.com/blog/MapReduce%20Quick%20Explanation (Accessed on March 8, 2020).
# [10] ..

<a id='another_cell'></a>

![title](img/thumbs-up.png)